# Scan for TF binding motifs and generate base GRN

This notebook demonstrates how to scan for TF binding motifs. The base GRN will be generated by combining the ATAC-seq peaks and motif information.

Much of this code is from CellOracle tutorial: https://morris-lab.github.io/CellOracle.documentation/notebooks/02_motif_scan/02_atac_peaks_to_TFinfo_with_celloracle_20200801.html

### 0. Import libraries

In [12]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


import seaborn as sns

import os, sys, shutil, importlib, glob
from tqdm.notebook import tqdm

%config InlineBackend.figure_format = 'retina'
%matplotlib inline

plt.rcParams['figure.figsize'] = (15,7)
plt.rcParams["savefig.dpi"] = 600

In [13]:
import warnings
warnings.filterwarnings('ignore')
import celloracle as co
from celloracle import motif_analysis as ma
from celloracle.utility import save_as_pickled_object
co.__version__

'0.14.0'

### 1. Reference genome data preparation
Before starting the analysis, we need to make sure the reference genome data is installed with genomepy. If not, please install the correct reference genome using the instructions below.

genomepy installs reference genome data under home directory. But if you have installed or want to install reference genome to another specific location, please specify the place using genomes_dir argument.

In [14]:
ref_genome = "mm10"
# genomes_directory = "SET GENOMES DIRECTORY HERE"
genomes_directory = "/project/clark-saucerman/genomes"


genome_installation = ma.is_genome_installed(ref_genome=ref_genome,
                                             genomes_dir=genomes_directory)
print(ref_genome, "installation: ", genome_installation)

if not genome_installation:
    import genomepy
    genomepy.install_genome(name=ref_genome, provider="UCSC", genomes_dir=genomes_directory)
else:
    print(ref_genome, "is installed.")

mm10 installation:  True
mm10 is installed.


### 2. Load data
In this notebook, we explain how to make a base GRN.

Using the `processed_peak_file.csv` data file that was output from `6-tss-annotation.ipynb`

In [15]:
data_dir = "data/base_grn_outputs/E9/"
# wt_or_ko = "KO"
peaks = pd.read_csv(f"{data_dir}/processed_peak_file.csv", index_col=0)
peaks.head()

,peak_id,gene_short_name
0,chr10_100015519_100016019,Kitl
1,chr10_100487279_100487779,Tmtc3
2,chr10_100487943_100488443,Cep290
3,chr10_100487943_100488443,Tmtc3
4,chr10_100589013_100589513,4930430F08Rik


Here, the function below will check peak data format, including chromosome name and peak length.

In [16]:
peaks = ma.check_peak_format(peaks, ref_genome, genomes_dir=genomes_directory)

Peaks before filtering:  25905
Peaks with invalid chr_name:  0
Peaks with invalid length:  0
Peaks after filtering:  25905


You can choose to use either a custom TF binding reference or CellOracle’s default motifs during the motif analysis. If you would like to use our default motifs, you can continue to the next step without loading any additional data. I am using the default!!

### 3. Instantiate TFinfo object and search for TF binding motifs
The motif analysis module has a custom class, TFinfo. The TFinfo object executes the steps below.

1. Converts a peak data into a DNA sequences.
2. Scans the DNA sequences searching for TF binding motifs.
3. Post-processes the motif scan results.
4. Converts data into appropriate format. You can convert data into base-GRN. The base GRN data can be formatted as either a python dictionary or pandas dataframe. This output will be the final base GRN used in the GRN model construction step.

If your reference genome file are installed in non-default location, please speficy the location using genomes_dir.



In [17]:
# Instantiate TFinfo object
tfi = ma.TFinfo(peak_data_frame=peaks,
                ref_genome=ref_genome,
                genomes_dir=genomes_directory)

You can specify the TF binding motif data as follows.

tfi.scan(motifs=motifs)

If you do not specify the motifs or set motifs to None, the default motifs will be loaded automatically.

For mouse and human, “gimme.vertebrate.v5.0.” will be used as the default motifs.

For another species, the species-specific TF binding motif data extracted from CisBP ver2.0 will be used.

NEXT, scan for TF binding motifs. This will take a while!!

In [18]:
%%time
# Scan motifs. !!CAUTION!! This step may take several hours if you have many peaks!
tfi.scan(fpr=0.02,
         motifs=None,  # If you enter None, default motifs will be loaded.
         verbose=True)

# Save tfinfo object
tfi.to_hdf5(file_path=f"{data_dir}/tfinfo.celloracle.tfinfo")

No motif data entered. Loading default motifs for your species ...
 Default motif for vertebrate: gimme.vertebrate.v5.0. 
 For more information, please see https://gimmemotifs.readthedocs.io/en/master/overview.html 

Initiating scanner... 

Calculating FPR-based threshold. This step may take substantial time when you load a new ref-genome. It will be done quicker on the second time. 

Motif scan started .. It may take long time.



scanning:   0%|          | 0/24574 [00:00<?, ? sequences/s]

CPU times: user 12min 41s, sys: 24.5 s, total: 13min 6s
Wall time: 14min 10s


In [19]:
tfi.scanned_df.head()

,seqname,motif_id,factors_direct,factors_indirect,score,pos,strand
0,chr10_100015519_100016019,GM.5.0.Mixed.0001,,"SRF, EGR1",7.904716,253,1
1,chr10_100015519_100016019,GM.5.0.Nuclear_receptor.0002,NR2C2,"Nr2c2, NR2C2",9.063828,221,-1
2,chr10_100015519_100016019,GM.5.0.Myb_SANT.0001,"MYBL2, MYBL1","MYBL2, MYBL1",6.630190,441,-1
3,chr10_100015519_100016019,GM.5.0.C2H2_ZF.0003,ZBTB6,"Zbtb6, ZBTB6",8.347882,425,1
4,chr10_100015519_100016019,GM.5.0.C2H2_ZF.0006,"Sp3, KLF5, SP2, SP3, SP1, KLF4, KLF12","Sp9, Klf5, IRF1, Sp8, Klf6, SP7, SP1, SP6, Sp5...",9.288466,156,-1


### 4. Filtering motifs

In [20]:
# Reset filtering
tfi.reset_filtering()

# Do filtering
tfi.filter_motifs_by_score(threshold=10)

# Format post-filtering results.
tfi.make_TFinfo_dataframe_and_dictionary(verbose=True)

Filtering finished: 7445888 -> 1557816
1. Converting scanned results into one-hot encoded dataframe.


  0%|          | 0/24574 [00:00<?, ?it/s]

2. Converting results into dictionaries.


  0%|          | 0/16080 [00:00<?, ?it/s]

  0%|          | 0/1090 [00:00<?, ?it/s]

### 5. Get final base GRN

In [21]:
df = tfi.to_dataframe()
df.head()

,peak_id,gene_short_name,9430076c15rik,Ac002126.6,Ac012531.1,Ac226150.2,Afp,Ahr,Ahrr,Aire,...,Znf784,Znf8,Znf816,Znf85,Zscan10,Zscan16,Zscan22,Zscan26,Zscan31,Zscan4
0,chr10_100015519_100016019,Kitl,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,chr10_100487279_100487779,Tmtc3,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,chr10_100487943_100488443,Cep290,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,chr10_100487943_100488443,Tmtc3,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,chr10_100589013_100589513,4930430F08Rik,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### 6. Save results
We will use this information when constructing the GRN models later. Save the results.

In [22]:
# Save result as a dataframe
df = tfi.to_dataframe()
df.to_parquet(f"{data_dir}/base_GRN_dataframe.parquet")